In [3]:
from IPython.display import display
import re

import cplex

In [4]:
# First, we can import some functions so we can use the model
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
from cameo.strain_design import pathway_prediction

# Second, we can read the GEM and save it as ‘model’
model = read_sbml_model('ecYeastGEM_prot.xml')

# Thrid, we can show general information about the loaded model
model

Name,M_ecYeastGEM_prot_v8__46__3__46__4
Memory address,0x07f06e14a0470
Number of metabolites,4180
Number of reactions,8144
Number of groups,0
Objective expression,-1.0*prot_pool_exchange + 1.0*prot_pool_exchange_reverse_c813a
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


In [12]:
model.reactions.get_by_id("prot_pool_exchange")

Reaction identifier,prot_pool_exchange
Name,prot_pool_exchange
Memory address,0x07f1271bef080
Stoichiometry,--> prot_pool[c] --> prot_pool [cytoplasm]
GPR,
Lower bound,0.0
Upper bound,0.0936065695352742


In [39]:
for reaction in model.reactions.query('zymosterol', 'name'):
     print(reaction.id)

r_1278
r_2106
r_2107
r_3553
r_3720
r_1278_REV
r_2106_REV
r_2107_REV
r_3553_REV
r_3720_REV
r_0130No1
r_0135No1
r_0235No1
r_0236No1
r_0237No1
r_0241No1
r_0130_REVNo1
r_0135_REVNo1


In [14]:
model.optimize()

,fluxes,reduced_costs
r_0006,0.026995,0.000000e+00
r_0070,0.000000,-1.355253e-19
r_0094,0.000000,-1.084202e-19
r_0099,0.000000,0.000000e+00
r_0200,0.000000,-4.365330e-04
...,...,...
draw_prot_Q3E790,0.000000,0.000000e+00
draw_prot_Q99190,0.000000,0.000000e+00
draw_prot_Q99288,0.000000,0.000000e+00
draw_prot_Q99321,0.000000,0.000000e+00


From this, we can see that the model has 4180 metabolites, 8144 reactions, and 14 compartments.

To get a quick overview of the model and its throughness, we can use Memote (source) to analyze the GEM and score how good it currently looks. Hopefully, our changes will improve the model, or at least not make it any worse. 

In [15]:
## NOTE: This field is commented out as it takes ~1 hour to compute every time

# %%time
# !memote report snapshot ecYeastGEM_prot.xml --filename ecYeastGEM_prot.html

The file "ecYeastGEM_prot.html" is the memote analysis.
From this analysis, we can see that the total score is 16%, which is quite low. However, the tests for "Charge Balance", "Metabolite Connectivity", and a few of the reaction annotations get a score of 100%. This indicates that (Write something clever here.)

Based on the KEGG database, one way to produce 7-dehydrocholesterol is throught the [human biosynthesis pathway](https://www.genome.jp/kegg-bin/show_module?hsa_M00101+1718). Our yeast model can natively produce zymosterol, and from this we can produce 7-dehydrocholesterol.

To get the pathway from zymosterol to 7-dehydrocholesterol, we will need three reactions: [R07498](https://www.genome.jp/dbget-bin/www_bget?R07498), [R03353](https://www.genome.jp/dbget-bin/www_bget?R03353), and [R07215](https://www.genome.jp/dbget-bin/www_bget?R07215).
We will now add these to the model in turn.

In [5]:
# First reaction

new_reaction1 = Reaction('R07498')  # The first of the three reactions from above

# R07498 uses:
## Zymosterol + NADPH + H+ <=> 5alpha-Cholest-8-en-3beta-ol + NADP+
## We can find these as: (Note: All of these are in the cytoplasm)
### Zymosterol = s_1569[c]
### NADPH      = s_1212[c]
### H+         = s_0794[c]
### 
### 5alpha-Cholest-8-en-3beta-ol = Not yet defined
### NADP+                        = s_1207[c]

# Since 5alpha-Cholest-8-en-3beta-ol does not exist, we will have to define it.
alphaCholest = Metabolite(id='5alpha-Cholest-8-en-3beta-ol_c', name='5alpha-Cholest-8-en-3beta-ol', compartment='c')


# We can now define the reaction stoichiometry
new_reaction1.add_metabolites({
                            model.metabolites.get_by_id("s_1569[c]"): -1,
                            model.metabolites.get_by_id("s_1212[c]"): -1,
                            model.metabolites.get_by_id("s_0794[c]"): -1,
                            alphaCholest: 1, # our newly created metabolites
                            model.metabolites.get_by_id("s_1207[c]"): 1
                             })

model.add_reactions([new_reaction1])

## Since the reaction can go both ways, we will set the bounds of the reaction to -1000 and 1000
#model.reactions.R07498.bounds = -1000, 1000

In [6]:

r_5a_exp = model.add_boundary(model.metabolites.get_by_id('5alpha-Cholest-8-en-3beta-ol_c'), type='demand')

with model:
    model.objective = r_5a_exp
    sol = model.optimize()
    print(sol.objective_value)

3.0861404508308876e-09


In [7]:
sol

,fluxes,reduced_costs
r_0006,2.699450e-02,0.0
r_0070,0.000000e+00,0.0
r_0094,0.000000e+00,0.0
r_0099,0.000000e+00,0.0
r_0200,0.000000e+00,0.0
...,...,...
draw_prot_Q99288,0.000000e+00,0.0
draw_prot_Q99321,0.000000e+00,0.0
prot_pool_exchange,8.499896e-02,0.0
R07498,3.086140e-09,0.0


In [21]:

with model:
    model.objective = model.reactions.get_by_id("r_0237No1")
    sol = model.optimize()
    print(sol.objective_value)

0.003980492411860787


In [19]:
model.reactions.get_by_id("DM_s_1569[c]")

Reaction identifier,DM_s_1569[c]
Name,zymosterol [cytoplasm] demand
Memory address,0x07f06dcd53668
Stoichiometry,s_1569[c] --> zymosterol [cytoplasm] -->
GPR,
Lower bound,0
Upper bound,1000.0


In [8]:
sol.fluxes[sol.fluxes != 0.]

r_0006                               2.699450e-02
r_2114                               6.069393e-04
r_1099                               3.021348e-02
r_1110                               6.861314e+00
r_1115                               5.929479e-01
                                         ...     
draw_prot_Q12362                     2.805519e-07
draw_prot_Q12676                     1.450320e-07
prot_pool_exchange                   8.499896e-02
R07498                               3.086140e-09
DM_5alpha-Cholest-8-en-3beta-ol_c    3.086140e-09
Name: fluxes, Length: 906, dtype: float64

In [9]:
sol.fluxes.index[sol.fluxes.argmax()]

'arm_r_0438'

In [10]:
model.metabolites.get_by_id("s_1569[c]")

Metabolite identifier,s_1569[c]
Name,zymosterol [cytoplasm]
Memory address,0x07f06e07d02b0
Formula,C27H44O
Compartment,c
In 8 reaction(s),"R07498, r_1682, r_0237No1, r_2107, r_0986No1, r_3553, r_3553_REV, r_2107_REV"


In [16]:
model.reactions.get_by_id("r_3553")

Reaction identifier,r_3553
Name,"zymosterol transport, cytoplasm-ER membrane"
Memory address,0x07f06dd5b8748
Stoichiometry,s_1569[c] --> s_3444[erm] zymosterol [cytoplasm] --> zymosterol [endoplasmic reticulum membrane]
GPR,
Lower bound,0.0
Upper bound,inf


In [31]:
model2 = read_sbml_model('yeastGEM.xml')

In [25]:
model2.metabolites.get_by_id("s_1569[c]")

Metabolite identifier,s_1569[c]
Name,zymosterol [cytoplasm]
Memory address,0x07f06d5b77550
Formula,C27H44O
Compartment,c
In 5 reaction(s),"r_3553, r_0986, r_0237, r_2107, r_1682"


In [32]:
# First reaction

new_reaction1 = Reaction('R07498')  # The first of the three reactions from above

# R07498 uses:
## Zymosterol + NADPH + H+ <=> 5alpha-Cholest-8-en-3beta-ol + NADP+
## We can find these as: (Note: All of these are in the cytoplasm)
### Zymosterol = s_1569[c]
### NADPH      = s_1212[c]
### H+         = s_0794[c]
### 
### 5alpha-Cholest-8-en-3beta-ol = Not yet defined
### NADP+                        = s_1207[c]

# Since 5alpha-Cholest-8-en-3beta-ol does not exist, we will have to define it.
alphaCholest = Metabolite(id='5alpha-Cholest-8-en-3beta-ol_c', name='5alpha-Cholest-8-en-3beta-ol', compartment='c')


# We can now define the reaction stoichiometry
new_reaction1.add_metabolites({
                            model.metabolites.get_by_id("s_1569[c]"): -1,
                            model.metabolites.get_by_id("s_1212[c]"): -1,
                            model.metabolites.get_by_id("s_0794[c]"): -1,
                            alphaCholest: 1, # our newly created metabolites
                            model.metabolites.get_by_id("s_1207[c]"): 1
                             })

model2.add_reactions([new_reaction1])

## Since the reaction can go both ways, we will set the bounds of the reaction to -1000 and 1000
#model.reactions.R07498.bounds = -1000, 1000

In [27]:

r_5a_exp = model2.add_boundary(model.metabolites.get_by_id('5alpha-Cholest-8-en-3beta-ol_c'), type='demand')

with model2:
    model2.objective = r_5a_exp
    sol = model2.optimize()
    print(sol.objective_value)

0.04122405543594532


In [33]:
# Second reaction

new_reaction2 = Reaction('R03353')  # The second of the three reactions from above

# R03353 uses:
## Lathosterol <=> 5alpha-Cholest-8-en-3beta-ol
## We can find these as: (Note: All of these are in the cytoplasm)
### 5alpha-Cholest-8-en-3beta-ol = alphaCholest
###
### Lathosterol                  = Not yet defined

# Since Lathosterol does not exist, we will have to define it.
Lathosterol = Metabolite(id='Lathosterol_c', name='Lathosterol', compartment='c')


# We can now define the reaction stoichiometry
new_reaction2.add_metabolites({
                            alphaCholest: -1, # The metabolite from above
                            Lathosterol: 1    # our newly created metabolites
                             })

model2.add_reactions([new_reaction2])

## Since the reaction can go both ways, we will set the bounds of the reaction to -1000 and 1000
#model.reactions.R03353.bounds = -1000, 1000

In [34]:

r_la_exp = model2.add_boundary(model.metabolites.get_by_id('Lathosterol_c'), type='demand')

with model2:
    model2.objective = r_la_exp
    sol = model2.optimize()
    print(sol.objective_value)

KeyError: 'Lathosterol_c'

In [29]:
# Third reaction

new_reaction3 = Reaction('R07215')  # The third of the three reactions from above

## R07215 uses:
## Lathosterol + 2 Ferrocytochromeb5 + Oxygen + 2 H+ <=> 7-Dehydrocholesterol + 2 Ferricytochrome b5 + 2 H2O
## We can find these as: (Note: All of these are in the cytoplasm, except for the cytochromes, which are in the ER)
### Lathosterol          = Lathosterol
### Ferrocytochrome b5   = s_3825[er]
### Oxygen               = s_1275[c]
### H+                   = s_0794[c]
###
### 7-Dehydrocholesterol = Not yet defined
### Ferricytochrome b5   = s_3824[er]
### H2O                  = s_0803[c]



# Since 7-Dehydrocholesterol does not exist, we will have to define it.
Dehydrocholesterol = Metabolite(id='7-Dehydrocholesterol_c', name='7-Dehydrocholesterol', compartment='c')


new_reaction3.add_metabolites({
                            Lathosterol: -1, # The metabolite we created above
                            model.metabolites.get_by_id("s_3825[er]"): -2,
                            model.metabolites.get_by_id("s_1275[c]"): -1,
                            model.metabolites.get_by_id("s_0794[c]"): -2,
                            Dehydrocholesterol: 1, # our newly created metabolites
                            model.metabolites.get_by_id("s_3824[er]"): 2,
                            model.metabolites.get_by_id("s_0803[c]"): 2
                             })

model2.add_reactions([new_reaction3])

## Since the reaction can go both ways, we will set the bounds of the reaction to -1000 and 1000
#model.reactions.R07215.bounds = -1000, 1000

In [32]:
model.metabolites.get_by_id("7-Dehydrocholesterol_c")

Metabolite identifier,7-Dehydrocholesterol_c
Name,7-Dehydrocholesterol
Memory address,0x07f124e7fc7b8
Formula,None
Compartment,c
In 2 reaction(s),"DM_7-Dehydrocholesterol_c, R07215"


In [34]:
model.reactions.get_by_id("DM_7-Dehydrocholesterol_c")

Reaction identifier,DM_7-Dehydrocholesterol_c
Name,7-Dehydrocholesterol demand
Memory address,0x07f1344189780
Stoichiometry,7-Dehydrocholesterol_c --> 7-Dehydrocholesterol -->
GPR,
Lower bound,0
Upper bound,1000.0


In [4]:
model.reactions.R07498

Reaction identifier,R07498
Name,
Memory address,0x07f2104fa4c50
Stoichiometry,s_0794[c] + s_1212[c] + s_1569[c] --> 5alpha-Cholest-8-en-3beta-ol_c + s_1207[c] H+ [cytoplasm] + NADPH [cytoplasm] + zymosterol [cytoplasm] --> 5alpha-Cholest-8-en-3beta-ol + NADP(+) [cytoplasm]
GPR,
Lower bound,0.0
Upper bound,1000.0


In [9]:
model.metabolites.get_by_id("7-Dehydrocholesterol_c")

Metabolite identifier,7-Dehydrocholesterol_c
Name,7-Dehydrocholesterol
Memory address,0x07f2104fb1588
Formula,None
Compartment,c
In 2 reaction(s),"DM_7-Dehydrocholesterol_c, R07215"


In [10]:
model.reactions.R07215

Reaction identifier,R07215
Name,
Memory address,0x07f2104fb1518
Stoichiometry,Lathosterol_c + 2 s_0794[c] + s_1275[c] + 2 s_3825[er] --> 7-Dehydrocholesterol_c + 2 s_0803[c] + 2 s_3824[er] Lathosterol + 2 H+ [cytoplasm] + oxygen [cytoplasm] + 2 Ferrocytochrome b5 [endoplasmic reticulum] --> 7-Dehydrocholesterol + 2 H2O [cytoplasm] + 2 Ferricytochrome b5 [endoplasmic reticulum]
GPR,
Lower bound,0.0
Upper bound,1000.0


In [ ]:
model.metabolites.

In [30]:

r_7DH_export = model2.add_boundary(model2.metabolites.get_by_id('7-Dehydrocholesterol_c'), type='demand')

with model2:
    r_5a_exp.bounds = 0,0
    model2.objective = r_7DH_export
    sol = model2.optimize()
    print(sol.objective_value)

0.0


In [8]:

r_7DH_export = model.add_boundary(model.metabolites.get_by_id('7-Dehydrocholesterol_c'), type='demand')

with model:
    model.objective = r_7DH_export
    sol = model.optimize()
    print(sol.objective_value)

0.0


In [5]:
with model:
    
    model.objective = model.reactions.R07498
    sol = model.optimize()
    print(sol.objective_value)

0.0


In [51]:
sol

,fluxes,reduced_costs
r_0006,0.026995,0.0
r_0070,0.000000,0.0
r_0094,0.000000,0.0
r_0099,0.000000,0.0
r_0200,0.000000,0.0
...,...,...
prot_pool_exchange,0.093607,0.0
R0749,0.000000,0.0
R03353,0.000000,0.0
R07215,0.000000,0.0


In [57]:
sol.fluxes.index[sol.fluxes.argmax()]

'arm_r_0438'

In [58]:
model.reactions.arm_r_0438

Reaction identifier,arm_r_0438
Name,ferrocytochrome-c:oxygen oxidoreductase (arm)
Memory address,0x07f1273801f28
Stoichiometry,s_0710[m] + 1.266 s_0799[m] + 0.25 s_1278[m] --> pmet_r_0438[m] ferrocytochrome c [mitochondrion] + 1.266 H+ [mitochondrion] + 0.25 oxygen [mitochondrion] --> pmet_r_0438 [mitochondrion]
GPR,( Q0045 and Q0250 and Q0275 and YDL067C and YEL039C and YGL187C and YGL191W and YHR051W and YIL11...
Lower bound,0.0
Upper bound,inf


In [53]:
model.reactions.r_0237No1

Reaction identifier,r_0237No1
Name,C-3 sterol keto reductase (zymosterol) (No1)
Memory address,0x07f1273869d30
Stoichiometry,0.00038052 prot_Q12452[c] + s_0794[c] + s_1212[c] + s_1579[c] --> s_1207[c] + s_1569[c] 0.00038052 prot_Q12452 [cytoplasm] + H+ [cytoplasm] + NADPH [cytoplasm] + zymosterol intermediate 2 [cytoplasm] --> NADP(+) [cytoplasm] + zymosterol [cytoplasm]
GPR,YLR100W
Lower bound,0.0
Upper bound,inf


In [43]:
with model:
    model.objective = model.reactions.r_0237No1
    sol = model.optimize()
    print(sol.objective_value)

0.003980492411860787


In [44]:
sol

,fluxes,reduced_costs
r_0006,0.026995,0.0
r_0070,0.000000,0.0
r_0094,0.000000,0.0
r_0099,0.000000,0.0
r_0200,0.000000,0.0
...,...,...
prot_pool_exchange,0.093607,0.0
R0749,0.000000,-0.0
R03353,0.000000,-0.0
R07215,0.000000,-0.0


In [27]:
with model:
    model.objective = model.reactions.r_2111
    sol = model.optimize()
    print(sol.objective_value)

0.09998999999999818


In [63]:
model.optimize()

,fluxes,reduced_costs
r_0006,0.026995,0.000000e+00
r_0070,0.000000,-3.388132e-20
r_0094,0.000000,-2.523658e-03
r_0099,0.000000,0.000000e+00
r_0200,0.000000,-4.365330e-04
...,...,...
draw_prot_Q99321,0.000000,0.000000e+00
prot_pool_exchange,0.053511,0.000000e+00
R0749,0.000000,0.000000e+00
R03353,0.000000,0.000000e+00


## 4. Computer-Aided Cell Factory Engineering (<1500 words if Category II project; <500 words for Category I project)